# Package and Data Loading

In [ ]:
import numpy as np
import pandas as pd
from keras import layers
from keras import applications
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model, Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, AveragePooling2D, GlobalMaxPooling2D, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras.initializers import glorot_uniform
import matplotlib.pyplot as plt
import random
import os

In [ ]:
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
BATCH_SIZE=30
EPOCHS=50
train_dir = "../input/dogs-vs-cats/train/train"
test_dir = "../input/dogs-vs-cats/test1/test"

In [ ]:
train_filenames = os.listdir(train_dir)
train_lables = []
for filename in train_filenames:
    label = filename.split('.')[0]
    train_lables.append(label)
train = pd.DataFrame({
    'filename': train_filenames,
    'label': train_lables
})
train_df, validate_df = train_test_split(train, test_size=0.20)
validate_size = validate_df.shape[0]
train_size = train_df.shape[0]

# Image Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    train_dir, 
    x_col='filename',
    y_col='label',
    target_size=IMAGE_SIZE,
    class_mode='binary',
    batch_size=BATCH_SIZE
)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    train_dir, 
    x_col='filename',
    y_col='label',
    target_size=IMAGE_SIZE,
    class_mode='binary',
    batch_size=BATCH_SIZE
)

# ResNet 50

In [ ]:
import keras_resnet.models
shape, classes = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS),2
x = Input(shape)
ResNetmodel = keras_resnet.models.ResNet50(x,classes=classes)
ResNetmodel.compile("adam","sparse_categorical_crossentropy",["accuracy"])
ResNetmodel.summary()

In [ ]:
earlystop = EarlyStopping(patience=10)

ResNetfilepath = "/kaggle/working/ResNet-Original-Best.hdh5"
# Keep only a single checkpoint, the best over test accuracy.
ResNetcheckpoint = ModelCheckpoint(ResNetfilepath,
                            monitor='val_loss',
                            verbose=1,
                            save_best_only=True,
                            mode='auto',
                            period=1)
ResNetcallbacks = [earlystop, ResNetcheckpoint]

In [ ]:
ResNethistory = ResNetmodel.fit_generator(
    train_generator, 
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validate_size/BATCH_SIZE,
    steps_per_epoch=train_size/BATCH_SIZE,
    callbacks=ResNetcallbacks,
)

# VGG3 No Dropout

In [ ]:
model_VGG3 = Sequential()

model_VGG3.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
model_VGG3.add(MaxPooling2D(pool_size=(2, 2)))

model_VGG3.add(Conv2D(64, (3, 3), activation='relu'))
model_VGG3.add(MaxPooling2D(pool_size=(2, 2)))

model_VGG3.add(Conv2D(128, (3, 3), activation='relu'))
model_VGG3.add(MaxPooling2D(pool_size=(2, 2)))


model_VGG3.add(Flatten())

model_VGG3.add(Dense(128, activation='relu'))
model_VGG3.add(Dense(1, activation='sigmoid'))

model_VGG3.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model_VGG3.summary()

In [ ]:
earlystop_VGG3 = EarlyStopping(patience=10)

filepath_VGG3 = "/kaggle/working/VGG-NoDropout-Best.hdh5"
# Keep only a single checkpoint, the best over test accuracy.
checkpoint_VGG3 = ModelCheckpoint(filepath_VGG3,
                            monitor='val_loss',
                            verbose=1,
                            save_best_only=True,
                            mode='auto',
                            period=1)
callbacks_VGG3 = [earlystop_VGG3, checkpoint_VGG3]

In [ ]:
history_VGG3 = model_VGG3.fit_generator(
    train_generator, 
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validate_size/BATCH_SIZE,
    steps_per_epoch=train_size/BATCH_SIZE,
    callbacks=callbacks_VGG3,
)

# VGG3 with Dropout 0.2

In [ ]:
model_VGG_020 = Sequential()

model_VGG_020.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
model_VGG_020.add(MaxPooling2D(pool_size=(2, 2)))
model_VGG_020.add(Dropout(0.2))
model_VGG_020.add(Conv2D(64, (3, 3), activation='relu'))
model_VGG_020.add(MaxPooling2D(pool_size=(2, 2)))
model_VGG_020.add(Dropout(0.2))
model_VGG_020.add(Conv2D(128, (3, 3), activation='relu'))
model_VGG_020.add(MaxPooling2D(pool_size=(2, 2)))
model_VGG_020.add(Dropout(0.2))

model_VGG_020.add(Flatten())

model_VGG_020.add(Dense(128, activation='relu'))
model_VGG_020.add(Dropout(0.5))
model_VGG_020.add(Dense(1, activation='sigmoid'))

model_VGG_020.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model_VGG_020.summary()

In [ ]:
earlystop_VGG_020 = EarlyStopping(patience=10)

filepath_VGG_020 = "/kaggle/working/VGG-Dropout020-Best.hdh5"
# Keep only a single checkpoint, the best over test accuracy.
checkpoint_VGG_020 = ModelCheckpoint(filepath_VGG_020,
                            monitor='val_loss',
                            verbose=1,
                            save_best_only=True,
                            mode='auto',
                            period=1)
callbacks_VGG_020 = [earlystop_VGG_020, checkpoint_VGG_020]

In [ ]:
history_VGG_020 = model_VGG_020.fit_generator(
    train_generator, 
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validate _size/BATCH_SIZE,
    steps_per_epoch=train_size/BATCH_SIZE,
    callbacks=callbacks_VGG_020,
)

# VGG3 with Dropout 0.3

In [ ]:
model_VGG_030 = Sequential()

model_VGG_030.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
model_VGG_030.add(MaxPooling2D(pool_size=(2, 2)))
model_VGG_030.add(Dropout(0.3))
model_VGG_030.add(Conv2D(64, (3, 3), activation='relu'))
model_VGG_030.add(MaxPooling2D(pool_size=(2, 2)))
model_VGG_030.add(Dropout(0.3))
model_VGG_030.add(Conv2D(128, (3, 3), activation='relu'))
model_VGG_030.add(MaxPooling2D(pool_size=(2, 2)))
model_VGG_030.add(Dropout(0.3))

model_VGG_030.add(Flatten())

model_VGG_030.add(Dense(128, activation='relu'))
model_VGG_030.add(Dropout(0.5))
model_VGG_030.add(Dense(1, activation='sigmoid'))

model_VGG_030.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model_VGG_030.summary()

In [ ]:
earlystop_VGG_030 = EarlyStopping(patience=10)

filepath_VGG_030 = "/kaggle/working/VGG-Dropout030-Best.hdh5"
# Keep only a single checkpoint, the best over test accuracy.
checkpoint_VGG_030 = ModelCheckpoint(filepath_VGG_030,
                            monitor='val_loss',
                            verbose=1,
                            save_best_only=True,
                            mode='auto',
                            period=1)
callbacks_VGG_030 = [earlystop_VGG_030, checkpoint_VGG_030]

In [ ]:
history_VGG_030 = model_VGG_030.fit_generator(
    train_generator, 
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validate_size/BATCH_SIZE,
    steps_per_epoch=train_size/BATCH_SIZE,
    callbacks=callbacks_VGG_030,
)

# VGG3 with Dropout 0.2 and Batch Normalization Batch Size 30

In [ ]:
model_VGG_020_Nor = Sequential()

model_VGG_020_Nor.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
model_VGG_020_Nor.add(BatchNormalization())
model_VGG_020_Nor.add(MaxPooling2D(pool_size=(2, 2)))
model_VGG_020_Nor.add(Dropout(0.2))
model_VGG_020_Nor.add(Conv2D(64, (3, 3), activation='relu'))
model_VGG_020_Nor.add(BatchNormalization())
model_VGG_020_Nor.add(MaxPooling2D(pool_size=(2, 2)))
model_VGG_020_Nor.add(Dropout(0.2))
model_VGG_020_Nor.add(Conv2D(128, (3, 3), activation='relu'))
model_VGG_020_Nor.add(BatchNormalization())
model_VGG_020_Nor.add(MaxPooling2D(pool_size=(2, 2)))
model_VGG_020_Nor.add(Dropout(0.2))

model_VGG_020_Nor.add(Flatten())

model_VGG_020_Nor.add(Dense(128, activation='relu'))
model_VGG_020_Nor.add(BatchNormalization())
model_VGG_020_Nor.add(Dropout(0.5))
model_VGG_020_Nor.add(Dense(1, activation='sigmoid'))

model_VGG_020_Nor.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model_VGG_020_Nor.summary()

In [ ]:
earlystop_VGG_020_Nor = EarlyStopping(patience=10)

filepath_VGG_020_Nor = "/kaggle/working/VGG-Dropout025-Nor-Batch30-Best.hdh5"
# Keep only a single checkpoint, the best over test accuracy.
checkpoint_VGG_020_Nor = ModelCheckpoint(filepath_VGG_020_Nor,
                            monitor='val_loss',
                            verbose=1,
                            save_best_only=True,
                            mode='auto',
                            period=1)
callbacks_VGG_020_Nor = [earlystop_VGG_020_Nor, checkpoint_VGG_020_Nor]

In [ ]:
history_VGG_020_Nor = model_VGG_020_Nor.fit_generator(
    train_generator, 
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validate_size/BATCH_SIZE,
    steps_per_epoch=train_size/BATCH_SIZE,
    callbacks=callbacks_VGG_020_Nor,
)

# VGG3 with Dropout 0.3 and Batch Nornalization

In [ ]:
model_VGG_030_Nor = Sequential()

model_VGG_030_Nor.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
model_VGG_030_Nor.add(BatchNormalization())
model_VGG_030_Nor.add(MaxPooling2D(pool_size=(2, 2)))
model_VGG_030_Nor.add(Dropout(0.3))
model_VGG_030_Nor.add(Conv2D(64, (3, 3), activation='relu'))
model_VGG_030_Nor.add(BatchNormalization())
model_VGG_030_Nor.add(MaxPooling2D(pool_size=(2, 2)))
model_VGG_030_Nor.add(Dropout(0.3))
model_VGG_030_Nor.add(Conv2D(128, (3, 3), activation='relu'))
model_VGG_030_Nor.add(BatchNormalization())
model_VGG_030_Nor.add(MaxPooling2D(pool_size=(2, 2)))
model_VGG_030_Nor.add(Dropout(0.3))

model_VGG_030_Nor.add(Flatten())

model_VGG_030_Nor.add(Dense(128, activation='relu'))
model_VGG_030_Nor.add(BatchNormalization())
model_VGG_030_Nor.add(Dropout(0.5))
model_VGG_030_Nor.add(Dense(1, activation='sigmoid'))

model_VGG_030_Nor.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model_VGG_030_Nor.summary()

In [ ]:
earlystop_VGG_030_Nor = EarlyStopping(patience=10)

filepath_VGG_030_Nor = "/kaggle/working/VGG-Dropout030_Nor-Best.hdh5"
# Keep only a single checkpoint, the best over test accuracy.
checkpoint_VGG_030_Nor = ModelCheckpoint(filepath_VGG_030_Nor,
                            monitor='val_loss',
                            verbose=1,
                            save_best_only=True,
                            mode='auto',
                            period=1)
callbacks_VGG_030_Nor = [earlystop_VGG_030_Nor, checkpoint_VGG_030_Nor]

In [ ]:
history_VGG_030_Nor = model_VGG_030_Nor.fit_generator(
    train_generator, 
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validate_size/BATCH_SIZE,
    steps_per_epoch=train_size/BATCH_SIZE,
    callbacks=callbacks_VGG_030_Nor,
)

# VGG 16

In [ ]:
model_VGG16 = applications.VGG16(include_top=False, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT,IMAGE_CHANNELS))

for layer in model_VGG16.layers:
    layer.trainable = False
flat1 = Flatten()(model_VGG16.layers[-1].output)
class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
output = Dense(1, activation='sigmoid')(class1)
model_VGG16 = Model(inputs=model_VGG16.inputs, outputs=output)
model_VGG16.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model_VGG16.summary()

In [ ]:
earlystop_VGG16 = EarlyStopping(patience=10)

filepath_VGG16 = "/kaggle/working/VGG16-Original-Best.hdh5"
# Keep only a single checkpoint, the best over test accuracy.
checkpoint_VGG16 = ModelCheckpoint(filepath_VGG16,
                            monitor='val_loss',
                            verbose=1,
                            save_best_only=True,
                            mode='auto',
                            period=1)
callbacks_VGG16 = [earlystop_VGG16, checkpoint_VGG16]

In [ ]:
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
history_VGG16 = model_VGG16.fit_generator(
    train_generator, 
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validate_size/BATCH_SIZE,
    steps_per_epoch=train_size/BATCH_SIZE,
    callbacks=callbacks_VGG16,
)

# VGG 19

In [ ]:
model_VGG19 = applications.VGG19(include_top=False, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT,IMAGE_CHANNELS))

for layer in model_VGG19.layers:
    layer.trainable = False
flat1 = Flatten()(model_VGG19.layers[-1].output)
class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
output = Dense(1, activation='sigmoid')(class1)
model_VGG19 = Model(inputs=model_VGG19.inputs, outputs=output)
model_VGG19.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model_VGG19.summary()

In [ ]:
earlystop_VGG19 = EarlyStopping(patience=10)

filepath_VGG19 = "/kaggle/working/VGG19-Original-Best.hdh5"
# Keep only a single checkpoint, the best over test accuracy.
checkpoint_VGG19 = ModelCheckpoint(filepath_VGG19,
                            monitor='val_loss',
                            verbose=1,
                            save_best_only=True,
                            mode='auto',
                            period=1)
callbacks_VGG19 = [earlystop_VGG19, checkpoint_VGG19]

In [ ]:
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
history_VGG19 = model_VGG19.fit_generator(
    train_generator, 
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validate_size/BATCH_SIZE,
    steps_per_epoch=train_size/BATCH_SIZE,
    callbacks=callbacks_VGG19,
)